In [76]:
import numpy as np
import pandas as pd
import os
import RAKE
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from autocorrect import spell
from nltk.stem import WordNetLemmatizer
import summa

In [6]:
unrest = pd.read_csv('GDELT_2017/unrest_gdelt_2017_sentiment.csv')

In [12]:
unrest.drop(['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1'],axis=1,inplace=True)

In [75]:
unrest.head()

,content,day,event_code,lat,long,month,place,time,title,url,year,sentiment_pos,sentiment_neg,sentiment_neu,sentiment_compound,sentiment_textblob,date,content_nostop_lemmatized,keywords
0,Bringing to an end the musical chairs game for...,1.0,40.0,28.6000,77.2000,1.0,IN,"Updated: January 1, 2017 11:34:40 am","In Arunachal, CM Pema Khandu wins musical chai...",http://indianexpress.com/article/india/in-arun...,2017.0,0.137,0.038,0.825,0.9965,0.082525,1-1-2017,,
1,MUMBAI: Request for a seat share landed a yout...,1.0,180.0,19.6833,72.7500,1.0,IN,NaN,Mumbaikar brutally assaulted in Dahanu local o...,http://timesofindia.indiatimes.com/city/mumbai...,2017.0,0.036,0.176,0.788,-0.9881,-0.255785,1-1-2017,,
2,The CIA staff of the Mohali police on Saturday...,1.0,173.0,24.1492,78.9817,1.0,IN,"Published: January 1, 2017 5:00:13 am",Two held with 2 kg heroin worth Rs 5 crore in ...,http://indianexpress.com/article/cities/chandi...,2017.0,0.023,0.180,0.797,-0.9790,0.105556,1-1-2017,,
3,THE PREDICTION of snowfall on New Years Eve dr...,1.0,120.0,31.9167,77.2500,1.0,IN,"Published: January 1, 2017 5:02:08 am","Tourists descend in hope of snow, Shimla feels...",http://indianexpress.com/article/cities/chandi...,2017.0,0.052,0.035,0.913,0.8085,0.135787,1-1-2017,,
4,This was the year when old certainties crumble...,1.0,42.0,18.9750,72.8258,1.0,IN,"Updated: January 1, 2017 12:46:58 pm",The Future of the Future: EYE 2017 special issue,http://indianexpress.com/article/lifestyle/lif...,2017.0,0.086,0.051,0.863,0.9431,0.096080,1-1-2017,,


In [53]:
#unrest['content_nostop_lemmatized'] = ''
unrest['keywords'] = ''

In [17]:
wordnet_lemmatizer = WordNetLemmatizer()

In [81]:
#data = unrest.iloc[5]['content']
#unrest_test = unrest[0:1]
#from summa import keywords
#print(keywords.keywords(data))

In [27]:
#r_obj = RAKE.Rake(RAKE.SmartStopList())

In [70]:
#r_obj.run(data,minCharacters=3,maxWords=5,minFrequency=1)[0][0]

In [73]:
stop_words = set(nltk.corpus.stopwords.words('english')) 
r_obj = RAKE.Rake(RAKE.SmartStopList())
for i,row in unrest.iterrows():
    tokens = nltk.word_tokenize(row['content'])
    text = nltk.Text(tokens)
    # Remove single character words (a, i)
    words = [word for word in text if len(word) > 1]
    # Fix the spellings of the words
    words = [spell(word) for word in words]
    # Convert all words to lower case
    words = [word.lower() for word in words if word.isalpha()]
    # remove the stop words from the tokens (an, the, it, and)
    words = [word for word in words if word not in stop_words]
    # Lemmatize all words (all verb forms are condensed to their root word)
    words = [wordnet_lemmatizer.lemmatize(word) for word in words]
    words = [wordnet_lemmatizer.lemmatize(word,'v') for word in words]
    st = ''
    for word in words:
        st = st+str(word)+' '
    st = st.strip()
    unrest.set_value(i,'content_nostop_lemmatized',st)
    keywords = ''
    k2 = r_obj.run(data,minCharacters=3,maxWords=2,minFrequency=1)
    k3 = r_obj.run(data,minCharacters=3,maxWords=3,minFrequency=1)
    k4 = r_obj.run(data,minCharacters=3,maxWords=4,minFrequency=1)
    k_list = []
    try:
        k_list.append(k2[0][0])  
        k_list.append(k3[0][0])
        k_list.append(k4[0][0])
    except:
        print('no keywords')
    try:
        k_list.append(k2[1][0])  
        k_list.append(k3[1][0])
        k_list.append(k4[1][0])
    except:
        print('no 2nd keywords')
    k_list = list(set(k_list))
    for k in k_list:
        keywords = keywords+str(k)+','
    unrest.set_value(i,'keywords',keywords)

In [78]:
#unrest.to_csv('gdelt_keywords_data.csv')